## Setup

In [1]:
!pip install wandb
!pip install transformers[deepspeed]

In [2]:
from datasets import Dataset,DatasetDict
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
from transformers import AutoModelForSequenceClassification,AutoTokenizer, AutoConfig
from transformers import TrainingArguments, Trainer
import torch
import os
import random
from transformers import EvalPrediction, TrainerCallback
import torch
from sklearn.metrics import mean_squared_error
import gc
import transformers
import wandb

"""
%%bash
cat <<'EOT' > ds_config_zero3.json
{
    "fp16": {
        "enabled": "auto",
        "loss_scale": 0,
        "loss_scale_window": 1000,
        "initial_scale_power": 16,
        "hysteresis": 2,
        "min_loss_scale": 1
    },

    "optimizer": {
        "type": "AdamW",
        "params": {
            "lr": "auto",
            "betas": "auto",
            "eps": "auto",
            "weight_decay": "auto"
        }
    },

    "scheduler": {
        "type": "WarmupLR",
        "params": {
            "warmup_min_lr": "auto",
            "warmup_max_lr": "auto",
            "warmup_num_steps": "auto"
        }
    },

    "zero_optimization": {
        "stage": 3,
        "offload_optimizer": {
            "device": "cpu",
            "pin_memory": true
        },
        "offload_param": {
            "device": "cpu",
            "pin_memory": true
        },
        "overlap_comm": true,
        "contiguous_gradients": true,
        "sub_group_size": 1e9,
        "reduce_bucket_size": "auto",
        "stage3_prefetch_bucket_size": "auto",
        "stage3_param_persistence_threshold": "auto",
        "stage3_max_live_parameters": 1e9,
        "stage3_max_reuse_distance": 1e9,
        "stage3_gather_16bit_weights_on_model_save": true
    },

    "gradient_accumulation_steps": "auto",
    "gradient_clipping": "auto",
    "steps_per_print": 2000,
    "train_batch_size": "auto",
    "train_micro_batch_size_per_gpu": "auto",
    "wall_clock_breakdown": false
}
EOT
"

In [3]:
wandb.login()

wandb: Currently logged in as: simveit. Use `wandb login --relogin` to force relogin


True

## Constants

In [4]:
%env WANDB_PROJECT=FP3

env: WANDB_PROJECT=FP3


In [5]:
# DeepSpeed requires a distributed environment even when only one process is used.
# This emulates a launcher in the notebook
#os.environ["MASTER_ADDR"] = "localhost"
#os.environ["MASTER_PORT"] = "9994"  # modify if RuntimeError: Address already in use
#os.environ["RANK"] = "0"
#os.environ["LOCAL_RANK"] = "0"
#os.environ["WORLD_SIZE"] = "1"
#os.environ["TOKENIZERS_PARALLELISM"] = "1"

In [6]:
BASE_PATH = "/notebooks"

In [7]:
CV_PATH = os.path.join(BASE_PATH, 'data/train/cv')

In [8]:
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

SEED = 28
seed_everything(seed=SEED)
MAX_LENGTH = 256

In [9]:
LABELS = ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']
id2label = {idx:label for idx, label in enumerate(LABELS)}
label2id = {label:idx for idx, label in enumerate(LABELS)}

## Functions

In [10]:
# Metric
def mcrmse(y_trues, y_preds):
    scores = []
    idxes = y_trues.shape[1]
    for i in range(idxes):
        y_true = y_trues[:,i]
        y_pred = y_preds[:,i]
        score = mean_squared_error(y_true, y_pred, squared=False) # RMSE
        scores.append(score)
    mcrmse_score = np.mean(scores)
    return mcrmse_score

    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    y_pred = torch.Tensor(predictions)
    y_true = labels
    mcrmse_acc = mcrmse(y_trues=y_true, y_preds=y_pred)
    # return as dictionary
    metrics = {'mcrmse': mcrmse_acc}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [11]:
from torch import nn

class CustomTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get('logits')
        loss_fct = nn.SmoothL1Loss(reduction='mean')
        loss = loss_fct(logits.squeeze(), labels.squeeze())
        return (loss, outputs) if return_outputs else loss

In [12]:
def train_model(model_dir, out_dir, df_train, df_val, 
                hyperparams={'bs': 4, 'lr': 9e-6, 'ep': 1, 'hidden_dropout_prob': 0.0, 'attention_probs_dropout_prob': 0.0}, 
                save_model = False):
    transformers.logging.set_verbosity_error()
    # CREATE DATASETS
    dataset_train = Dataset.from_pandas(df_train)
    dataset_test = Dataset.from_pandas(df_val)
    tokenizer = AutoTokenizer.from_pretrained(model_dir)
    
    # training scores
    scores = []
    
    def preprocess_data(examples):
        # take a batch of texts
        text = examples["full_text"]
        # encode them
        encoding = tokenizer(text, truncation=True, padding=True, max_length=MAX_LENGTH)
        # add labels
        labels_batch = {k: examples[k] for k in examples.keys() if k in LABELS}
        # create numpy array of shape (batch_size, num_labels)
        labels_matrix = np.zeros((len(text), len(LABELS)))
        # fill numpy array
        for idx, label in enumerate(LABELS):
            labels_matrix[:, idx] = labels_batch[label]
        encoding["labels"] = labels_matrix.tolist()
        return encoding

    dataset_train_encoded = dataset_train.map(preprocess_data, batched=True, remove_columns=LABELS+["full_text","text_id"])
    dataset_test_encoded = dataset_test.map(preprocess_data, batched=True, remove_columns=LABELS+["full_text","text_id"])
    
    
    # MODEL
    config = AutoConfig.from_pretrained(model_dir, 
                                        num_labels=len(LABELS),
                                        id2label=id2label,
                                        label2id=label2id, 
                                        hidden_dropout_prob=hyperparams['hidden_dropout_prob'],
                                        attention_probs_dropout_prob = hyperparams['attention_probs_dropout_prob']
                                        )
    model = AutoModelForSequenceClassification.from_pretrained(model_dir, 
                                                               config = config)
    
    # ARGUMENTS
    args = TrainingArguments(
            output_dir=out_dir,
            evaluation_strategy = "steps",
            warmup_ratio = 0.1,
            learning_rate = hyperparams['lr'],
            eval_steps = 100, 
            num_train_epochs=hyperparams['ep'],
            lr_scheduler_type='cosine',
            load_best_model_at_end=True,
            per_device_train_batch_size=hyperparams['bs'],
            per_device_eval_batch_size=hyperparams['bs'],
            #save_strategy="no",
            save_total_limit = 1,
            metric_for_best_model="mcrmse",
            greater_is_better = False,
            report_to="wandb",  # enable logging to W&B
            run_name=out_dir,  # name of the W&B run (optional)
            #deepspeed="ds_config_zero3.json" #deepspeed
     )
    
    class SaveLogs(TrainerCallback):
        """
        A bare [`TrainerCallback`] that just prints the logs.
        """

        def on_log(self, args, state, control, logs=None, **kwargs):
            _ = logs.pop("total_flos", None)
            if state.is_local_process_zero:
                try:
                    scores.append(logs["eval_mcrmse"])
                except:
                    pass
    
    # TRAINER
    trainer = CustomTrainer(model,
                            args,
                            train_dataset=dataset_train_encoded,
                            eval_dataset=dataset_test_encoded,
                            tokenizer=tokenizer,
                            compute_metrics=compute_metrics,
                            callbacks = [SaveLogs]
    )
    trainer.train()
    trainer.save_model("trainer" + out_dir)
    wandb.finish()
    if save_model:
        pass
    else:
        del model
        gc.collect()
        return min(scores)
    return all_logs

In [13]:
#train_model("microsoft/deberta-v3-small", "out", pd.read_csv("data/train/cv/train_fold_0.csv"), pd.read_csv("data/train/cv/val_fold_0.csv"))

In [14]:
def train_cv_v2(model_dir, out_dir, fold_dir, hyperparams={'bs': 4, 'lr': 9e-6, 'ep': 1},
                kfolds=[0, 1, 2, 3, 4, 5], continue_training = True):
    scores = []
    for fold in kfolds:
        train_df = pd.read_csv(fold_dir + '/train_fold_' + str(fold) + '.csv')
        val_df = pd.read_csv(fold_dir + '/val_fold_' + str(fold) + '.csv')

        model_out_dir = out_dir + '/model_fold_' + str(fold)
        if continue_training:
            final_model_dir = model_dir + '/model_fold_' + str(fold) + '/best'
        else:
            final_model_dir = model_dir

        best_score = train_model(
            model_dir=model_dir,
            out_dir=model_out_dir,
            df_train=train_df,
            df_val=val_df,
            save_model=False,
            hyperparams=hyperparams,
          )
        scores.append(best_score)
    cv_score = np.mean(scores)
    return scores, cv_score

In [15]:
model_dir = "microsoft/deberta-v3-large"
fold_dir = BASE_PATH + "/data/train/cv"
out_dir = "DEBERTA-LARGE"
hyperparams = {'bs': 4, 'lr': 2e-5, 'ep': 3, 'hidden_dropout_prob': 0.0, 'attention_probs_dropout_prob': 0.0}
scores, cv_score = train_cv_v2(model_dir, out_dir, fold_dir, hyperparams)

/usr/local/lib/python3.9/dist-packages/transformers/convert_slow_tokenizer.py:434: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Parameter 'function'=<function train_model.<locals>.preprocess_data at 0x7ffa5b9bac10> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed onc

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3259
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 2445
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


***** Running Evaluation *****
  Num examples = 652
  Batch size = 4


...

{'eval_loss': 0.11227554827928543, 'eval_mcrmse': 0.4753783047199249, 'eval_runtime': 22.0779, 'eval_samples_per_second': 29.532, 'eval_steps_per_second': 7.383, 'epoch': 2.94}




Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from DEBERTA-LARGE/model_fold_5/checkpoint-2000 (score: 0.47765398025512695).
Saving model checkpoint to trainerDEBERTA-LARGE/model_fold_5
Configuration saved in trainerDEBERTA-LARGE/model_fold_5/config.json


{'train_runtime': 1769.0949, 'train_samples_per_second': 5.527, 'train_steps_per_second': 1.382, 'train_loss': 0.15810979831438124, 'epoch': 3.0}


Model weights saved in trainerDEBERTA-LARGE/model_fold_5/pytorch_model.bin
tokenizer config file saved in trainerDEBERTA-LARGE/model_fold_5/tokenizer_config.json
Special tokens file saved in trainerDEBERTA-LARGE/model_fold_5/special_tokens_map.json


eval/loss,▅█▂▂▃▂▃▂▁▁▂▁▂▂▂▂▁▁▁▁▁▁▁▁
eval/mcrmse,▆█▃▂▄▂▃▂▁▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
eval/runtime,█▂▄▁▇▁▁▁▆▁▁▂▁▂▂▅▃▂▁▃▁▆▁▂
eval/samples_per_second,▁▇▄█▂███▃██▇█▇▇▄▆▇█▆█▃█▇
eval/steps_per_second,▁▇▄█▂███▃██▇█▇▇▄▆▇█▆█▃█▇
train/epoch,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▇▇▇▇███
train/global_step,▁▁▂▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▇▇▇▇███
train/learning_rate,█▆▃▁
train/loss,█▂▂▁
train/total_flos,▁
train/train_loss,▁


In [16]:
print("scores = ", scores)
print("cv_score = ", cv_score)

scores =  [0.45068123936653137, 0.4638478457927704, 0.46915867924690247, 0.4597109258174896, 0.4574316740036011, 0.4748063385486603]
cv_score =  0.46260611712932587
